In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import function as F
from airbnbclass import *
from airbnb_pipeline import *

In [3]:
pd.set_option("display.max_colwidth",200)
pd.set_option("display.max_rows",999)
pd.set_option("display.max_columns",999)

In [18]:
import pickle
import json
import requests
import datetime

In [5]:
sf = Airbnb()
columns = ['id','price','availability_30','availability_60','reviews_per_month',
'number_of_reviews','last_review','host_since','minimum_nights',
'room_type','host_response_time', 'host_is_superhost',
'review_scores_rating','property_type', 
'neighbourhood_cleansed','bedrooms',
'calculated_host_listings_count','host_identity_verified',
'cleaning_fee','last_scraped','latitude','longitude','beds',
'cancellation_policy','access','description','notes','transit',
'instant_bookable','extra_people','maximum_nights','house_rules',
#            'calendar_updated',
]
listings = sf.load_data(columns_used=columns)

Reading: /Users/Alex/Documents/10_galvanize/galvanize/capstone/dsi-project-proposals/project/data/listings20170802.csv...
reviews20170802.csv (9057, 37)
(13831, 37)
Reading: /Users/Alex/Documents/10_galvanize/galvanize/capstone/dsi-project-proposals/project/data/listings20180110.csv...
reviews20180110.csv (6506, 37)
(20337, 37)
Reading: /Users/Alex/Documents/10_galvanize/galvanize/capstone/dsi-project-proposals/project/data/listings20180304.csv...
reviews20180304.csv (4804, 37)
(25141, 37)
Reading: /Users/Alex/Documents/10_galvanize/galvanize/capstone/dsi-project-proposals/project/data/listings20171101.csv...
reviews20171101.csv (8929, 37)
(34070, 37)
Reading: /Users/Alex/Documents/10_galvanize/galvanize/capstone/dsi-project-proposals/project/data/listings20180202.csv...
reviews20180202.csv (4727, 37)
(38797, 37)
Reading: /Users/Alex/Documents/10_galvanize/galvanize/capstone/dsi-project-proposals/project/data/listings20180406.csv...
reviews20180406.csv (4844, 37)
(43641, 37)
Reading: /

In [6]:
listings.columns

Index(['id', 'last_scraped', 'description', 'notes', 'transit', 'access',
       'house_rules', 'host_since', 'host_response_time', 'host_is_superhost',
       'host_identity_verified', 'neighbourhood_cleansed', 'latitude',
       'longitude', 'property_type', 'room_type', 'bedrooms', 'beds', 'price',
       'cleaning_fee', 'extra_people', 'minimum_nights', 'maximum_nights',
       'availability_30', 'availability_60', 'number_of_reviews',
       'last_review', 'review_scores_rating', 'instant_bookable',
       'cancellation_policy', 'calculated_host_listings_count',
       'reviews_per_month', 'last30', 'last60', 'last90', 'last120',
       'last180'],
      dtype='object')

In [7]:
listings = sf.filter_data(listings)

Filter 1: 12226, i.e. 9.43% listings removed.

FIlter 2 : 22401, i.e.17.27% listings removed.

35217 of listings to check
Remove 34320 listings. 97.45% of listings checked.  

3192 of listings to check
Remove 2573 listings. 80.61% of listings checked.  

3243 of listings to check
Remove 2282 listings. 70.37% of listings checked.  

3315 of listings to check
Remove 2135 listings. 64.40% of listings checked.  

3277 of listings to check
Remove 2137 listings. 65.21% of listings checked.  

3105 of listings to check
Remove 1713 listings. 55.17% of listings checked.  

2926 of listings to check
Remove 1443 listings. 49.32% of listings checked.  

2768 of listings to check
Remove 1342 listings. 48.48% of listings checked.  

2735 of listings to check
Remove 1190 listings. 43.51% of listings checked.  

2432 of listings to check
Remove 786 listings. 32.32% of listings checked.  

2223 of listings to check
Remove 750 listings. 33.74% of listings checked.  

2030 of listings to check
Remove 491

In [8]:
listings.shape

(43408, 37)

In [9]:
pipe = Pipeline([
    ('pre_select', PreselectColumns()),
    ('clean_data', DataType()),
    ('FeatureEnginner', FeatureEnginner()),
    ('PricePerBedroom', PricePerBedroom()),
    ('DropColumns', DropColumns()),
    ('Getdummies', Getdummies()),
    ('rf', RandomForestRegressor())
])
# m = p.fit(X,y)
# X_train_post = m.transform(X_train)
# X_test_post = m.transform(X_test)

In [10]:
param_grid = {"rf__n_estimators": [50],}
#     "rf__max_depth": [4,6,8],
#     "rf__max_features": ['auto', 'sqrt', 'log2']}

In [11]:
X = listings.copy()
y = X.pop('availability_30')
# X = X.reset_index()

cv_cutoff_date = pd.to_datetime('2017-11-01')
cv = -1*(listings.last_scraped < cv_cutoff_date).astype(int)
cross_val = PredefinedSplit(cv)

rmsle_scorer = make_scorer(mean_squared_log_error, greater_is_better=False)
grid = GridSearchCV(pipe, param_grid=param_grid, cv=cross_val, scoring=rmsle_scorer,verbose=1, n_jobs=-1)
grid.fit(X, y)


with open('rf_pipe.pkl','wb') as f:
        pickle.dump(grid, f)

print(grid.best_score_)
print(grid.best_params_)


Fitting 1 folds for each of 1 candidates, totalling 1 fits


[Parallel(n_jobs=-1)]: Done   1 out of   1 | elapsed:   27.1s finished


-0.365954069873
{'rf__n_estimators': 50}


In [ ]:
grid.predict(new_data)

In [ ]:
new_data = listings.iloc[[22]]

In [ ]:
k = y.values
k[22]

In [ ]:
new_data

In [ ]:
with open('rf_pipe.pkl','wb') as f:
        pickle.dump(grid, f)

In [ ]:
with open('rf_pipe.pkl', 'rb') as f:
        model = pickle.load(f)
model.predict(new_data)

In [ ]:
rmsle_scorer = make_scorer(mean_squared_log_error, greater_is_better=False)

In [ ]:
X_train, X_test, y_train, y_test = F.my_train_test_split(listings, '2017-11-01')
p = Pipeline([
    ('pre_select', PreselectColumns()),
    ('clean_data', DataType()),
    ('FeatureEnginner', FeatureEnginner()),
    ('PricePerBedroom', PricePerBedroom()),
    ('DropColumns', DropColumns()),
    ('Getdummies', Getdummies()),
#     ('rf', RandomForestRegressor())
])
mod = p.fit(X_train,y_train)
X_train_post = mod.transform(X_train)
X_test_post = mod.transform(X_test)


rf = RandomForestRegressor(n_estimators=100)
rf.fit(X_train_post,y_train)


y_pred = rf.predict(X_test_post)
print(mean_squared_log_error(y_test,y_pred))

neg_scorer = lambda y1, y2: mean_squared_log_error(y1, y2)*(-1)
feature_importance = F.permutation_importance(rf, X_test_post.values, y_test, scorer=neg_scorer)
series_feature_importance = pd.Series(feature_importance,index=X_train_post.columns.tolist())
feature_importance = series_feature_importance.copy()

F.plot_feature_importance(feature_importance)

In [ ]:
preselect_cols = list(set([
        # 'id',
        'price',
        'reviews_per_month',
        'number_of_reviews',
        # 'last_review',
        'host_since',
        # 'minimum_nights',
        'room_type',
        'host_response_time',
        # 'host_is_superhost',
        'review_scores_rating',
        'property_type',
        'neighbourhood_cleansed',
        'bedrooms',
        # 'calculated_host_listings_count',
        # 'host_identity_verified',
        'cleaning_fee',
        'last_scraped',
        'latitude',
        'longitude',
        # 'beds',
        # 'cancellation_policy',
        # 'access',
        # 'description',
        # 'notes','transit',
        'instant_bookable',
        'extra_people',
        'maximum_nights',
        # 'house_rules',
        ]))

In [ ]:
new_data = X_train.loc[8,preselect_cols].head(1)

In [ ]:
new_data.to_dict()

In [12]:
address = '44 Tehama St, San Francisco, CA 94105'

In [13]:
search_query = "https://maps.googleapis.com/maps/api/geocode/json?address="
search_query += address + '&key=' + 'keys'

#         print(search_query)
response = requests.get(search_query)
result = json.loads(response.text)


latitude = result['results'][0]['geometry']['location']['lat']
longitude = result['results'][0]['geometry']['location']['lng']
neighbourhood_cleansed = result['results'][0]['address_components'][2]['short_name']

In [15]:
print(latitude)
print(longitude)
print(neighbourhood_cleansed)

37.7876372
-122.3967284
South of Market


datetime.datetime(2018, 8, 7, 7, 52, 46, 353354)

In [36]:
new_data = {'bedrooms': {8: 3.0},
 'cleaning_fee': {8: '$150.00'},
 'extra_people': {8: '$0.00'},
 'host_response_time': {8: 'within an hour'},
 'host_since': {8: pd.to_datetime('2018-08-01')},
 'instant_bookable': {8: 'f'},
 'last_scraped': {8: pd.to_datetime('2018-08-06')},
 'latitude': {8: latitude},
 'longitude': {8: longitude},
 'maximum_nights': {8: 1125},
 'neighbourhood_cleansed': {8: 'Seacliff'},
 'number_of_reviews': {8: number_of_reviews},
 'price': {8: '$195.00'},
 'property_type': {8: 'Apartment'},
 'review_scores_rating': {8: 99.0},
 'reviews_per_month': {8: reviews_per_month},
 'room_type': {8: 'Entire home/apt'}}
new_data = pd.DataFrame(new_data)

,bedrooms,cleaning_fee,extra_people,host_response_time,host_since,instant_bookable,last_scraped,latitude,longitude,maximum_nights,neighbourhood_cleansed,number_of_reviews,price,property_type,review_scores_rating,reviews_per_month,room_type
8,3.0,$150.00,$0.00,within an hour,2018-08-01,f,2018-08-06,37.787637,-122.396728,1125,Seacliff,0,$195.00,Apartment,99.0,0,Entire home/apt


In [38]:
with open('rf_pipe.pkl', 'rb') as f:
        model = pickle.load(f)

array([ 16.32])

In [53]:
number_of_reviews = 0
last_scraped = pd.to_datetime('2018-01-01')
reviews_per_month=0
for month in range(1,13):
    new_data = {'bedrooms': {8: 3.0},
 'cleaning_fee': {8: '$50.00'},
 'extra_people': {8: '$0.00'},
 'host_response_time': {8: 'within an hour'},
 'host_since': {8: pd.to_datetime('2018-01-01')},
 'instant_bookable': {8: 't'},
 'last_scraped': {8: last_scraped},
 'latitude': {8: latitude},
 'longitude': {8: longitude},
 'maximum_nights': {8: 1125},
 'neighbourhood_cleansed': {8: neighbourhood_cleansed},
 'number_of_reviews': {8: number_of_reviews},
 'price': {8: '$195.00'},
 'property_type': {8: 'Apartment'},
 'review_scores_rating': {8: 99.0},
 'reviews_per_month': {8: reviews_per_month},
 'room_type': {8: 'Entire home/apt'}}
    print('month:', month)
    new_data = pd.DataFrame(new_data)
    result = model.predict(new_data)
    print('number of reviews cumulated:', round(number_of_reviews))
    last_scraped = pd.to_datetime('2018-08-07') + datetime.timedelta(month*30)
    number_of_reviews += (30 - int(result))/5/2
    reviews_per_month = number_of_reviews/month
    
    print('avaialable:',result)
    print('profit',(30-result)*150,'\n')

month: 1
number of reviews cumulated: 0
avaialable: [ 17.04]
profit [ 1944.] 

month: 2
number of reviews cumulated: 1
avaialable: [ 19.86]
profit [ 1521.] 

month: 3
number of reviews cumulated: 2
avaialable: [ 19.34]
profit [ 1599.] 

month: 4
number of reviews cumulated: 4
avaialable: [ 17.56]
profit [ 1866.] 

month: 5
number of reviews cumulated: 5
avaialable: [ 15.28]
profit [ 2208.] 

month: 6
number of reviews cumulated: 6
avaialable: [ 12.8]
profit [ 2580.] 

month: 7
number of reviews cumulated: 8
avaialable: [ 11.24]
profit [ 2814.] 

month: 8
number of reviews cumulated: 10
avaialable: [ 10.2]
profit [ 2970.] 

month: 9
number of reviews cumulated: 12
avaialable: [ 10.32]
profit [ 2952.] 

month: 10
number of reviews cumulated: 14
avaialable: [ 8.16]
profit [ 3276.] 

month: 11
number of reviews cumulated: 16
avaialable: [ 7.28]
profit [ 3408.] 

month: 12
number of reviews cumulated: 19
avaialable: [ 6.1]
profit [ 3585.] 

